In [3]:
# Dependencies and Libraries
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager
from pprint import pprint
import pandas as pd
import requests




In [6]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/yazminrodriguez/.wdm/drivers/chromedriver/mac64/91.0.4472.19/chromedriver] found in cache


### NASA Mars News
Finding the title and paragraph from: https://redplanetscience.com/

In [11]:
# Visit visitcostarica.herokuapp.com
url = "https://redplanetscience.com/"
browser.visit(url)
    

In [12]:
 # Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [17]:
# Get the news title and news paragraph
news_title = soup.find('div',class_='content_title')

#Paragraph
parag= soup.find('div',class_='article_teaser_body')



In [18]:
# Printing News title 
news_title.text


'Global Storms on Mars Launch Dust Towers Into the Sky'

In [19]:
#Printing paragraph
parag.text

'A Mars Dust Tower Stands Out Dust storms are common on Mars. But every decade or so, something unpredictable happens: a series of runaway storms break out, covering the entire planet in a dusty haze.'

In [24]:
# Save title and paragraph in dictionary mars
mars_info={}
mars_info["news_title"]= news_title.text
mars_info["news_paragraph"]= parag.text

In [25]:
#Print dict
mars_info

{'news_title': 'Global Storms on Mars Launch Dust Towers Into the Sky',
 'news_paragraph': 'A Mars Dust Tower Stands Out Dust storms are common on Mars. But every decade or so, something unpredictable happens: a series of runaway storms break out, covering the entire planet in a dusty haze.'}

### JPL Mars Space Images - Featured Image
Finding images from: https://spaceimages-mars.com/

In [26]:
# Visit visitcostarica.herokuapp.com
url =  'https://spaceimages-mars.com/'
browser.visit(url)

 # Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [27]:
# JPL Mars Space Images - Featured Image

relative_image_path = soup.find('img', class_='fade-in').get('src')

#Joining the URL of the page + the URL of the image

featured_image_url= 'https://spaceimages-mars.com/'+ relative_image_path

#Adding the image to Directory Mars
mars_info['mars_images']=relative_image_path


In [28]:
relative_image_path

'image/featured/mars3.jpg'

In [29]:

featured_image_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

In [30]:
#Printing the directory
mars_info

{'news_title': 'Global Storms on Mars Launch Dust Towers Into the Sky',
 'news_paragraph': 'A Mars Dust Tower Stands Out Dust storms are common on Mars. But every decade or so, something unpredictable happens: a series of runaway storms break out, covering the entire planet in a dusty haze.',
 'mars_images': 'image/featured/mars3.jpg'}

### Mars Facts
Finding a table from: https://galaxyfacts-mars.com/

In [32]:
# Visit visitcostarica.herokuapp.com
url =  'https://galaxyfacts-mars.com/'
browser.visit(url)

 # Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [35]:
# Use Panda's read_html to parse the url
mars_facts_table = pd.read_html(url, match='Equatorial Diameter')

# Find the mars facts DataFrame in the list of DataFrames and assign it to mars_df
mars_df = mars_facts_table[0]

# Assign the columns Category, Value
mars_df.columns = ['Category','Value']

# Set the index to the `Description` column without row indexing
mars_df.set_index('Category', inplace=True)

# Save html code to folder Assets
mars_df.to_html()

data = mars_df.to_dict(orient='records')

# Save Data
mars_info['facts_table']= mars_df

In [37]:
#Printing dict
mars_info

{'news_title': 'Global Storms on Mars Launch Dust Towers Into the Sky',
 'news_paragraph': 'A Mars Dust Tower Stands Out Dust storms are common on Mars. But every decade or so, something unpredictable happens: a series of runaway storms break out, covering the entire planet in a dusty haze.',
 'mars_images': 'image/featured/mars3.jpg',
 'facts_table':                                               Value
 Category                                           
 Equatorial Diameter:                       6,792 km
 Polar Diameter:                            6,752 km
 Mass:                 6.39 × 10^23 kg (0.11 Earths)
 Moons:                        2 ( Phobos & Deimos )
 Orbit Distance:            227,943,824 km (1.38 AU)
 Orbit Period:                  687 days (1.9 years)
 Surface Temperature:                   -87 to -5 °C
 First Record:                     2nd millennium BC
 Recorded By:                   Egyptian astronomers}

### Mars Hemispheres

Obtaining high resolution images for each of Mar's hemispheres.https://marshemispheres.com/


In [44]:
# Visit the website
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

url =  'https://marshemispheres.com/'
browser.visit(url)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/yazminrodriguez/.wdm/drivers/chromedriver/mac64/91.0.4472.19/chromedriver] found in cache


In [47]:
# Create a list for the full images url´s
hemispheres_img_urls = []

#Looping the webpage
for x in range(0,4):
    urls_retrieved= browser.links.find_by_partial_text('Hemisphere Enhanced')
    
    urls_retrieved[x].click()
    html= browser.html
    soup= bs(html,'html.parser')

    # Getting the url´s data
    title= soup.find('h2', class_='title').text
    img_url= soup.find('div', class_= 'downloads')
    img_url= img_url.find('a', target= '_blank')['href']
    img_url= 'https://marshemispheres.com/' + img_url
    
#Save the urls in the list of 'hemispheres_img_urls'
    hemispheres_image_urls.append({
            'Title': title,
            'Image URL': img_url
        })

# Going back of the browser
    browser.links.find_by_partial_text('Back').click()

browser.quit()

    # Save this images data into the mars_info dict
mars_info['hemisphere_images']= hemispheres_img_urls

return mars_info



ElementDoesNotExist: no elements could be found with link by partial text "Hemisphere Enhanced"

In [144]:
items=[]


#Loop through the items 
for i in items:
    #Store title
    title= i.find('h3').text
    
    #Store link that leads to full image website
    img_url= i.find('a', class_='open_toggle')['href']
    
    #Open link with the full image
    browser.visit(main_url_hemisphere + img_url)
    
    #HTML object of individual hemisphere information website
    img_url_html = browser.html
    
    #Parse HTML with bs for each hemisphere information website
    soup = bs(img_url_html, "html.parser")
    
    #Retrieve full image source
    full_img_url= main_url_hemisphere + soup.find('img',class_= 'wide-image')['src']
    
    #Append this retrieved info to the list of dicts.
    hemisphere_image_urls.append({'title':title, 'image_url':'full_img_url'})
 

NameError: name 'items' is not defined

In [ ]:
links = browser.find_by_css('a.product-item img')
   
#Print hemisphere list of dicts.
hemisphere_image_urls
    